In [ ]:
import torch
import torchaudio
import os
import requests
import numpy as np
import IPython
import matplotlib.pyplot as plt
from torchaudio.utils import download_asset
import pandas as pd

: 

In [ ]:
torch.random.manual_seed(0)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

: 

In [3]:
bundle = torchaudio.pipelines.WAV2VEC2_ASR_BASE_960H
model = bundle.get_model().to(device)

In [4]:
def create_audio_embeddings(features):
    audio_embed = torch.zeros([features[0].shape[1], features[0].shape[2]])
    for feat in features:
        audio_embed += torch.mean(feat, axis = 0)

    audio_embed = audio_embed.mean(axis = 0)
    return audio_embed

In [6]:
def extract_audio_feat(filename):
    embeddings = []
    i = 0
    for file in os.listdir(filename):
     # check the files which are end with specific extension
            # print path name of selected files
        AUDIO_FILE = os.path.join(filename, file)
        waveform, sample_rate = torchaudio.load(AUDIO_FILE)
        waveform = waveform.to(device)

        if sample_rate != bundle.sample_rate:
            waveform = torchaudio.functional.resample(waveform, sample_rate, bundle.sample_rate)

        with torch.inference_mode():
            features, _ = model.extract_features(waveform)

        embedding = create_audio_embeddings(features)
        embeddings.append(embedding)
        i = i+1
        print("File ",i)
            
#             if i == 5:
#                 break
            
            
    return embeddings
        

In [ ]:

for i, file in enumerate(os.listdir(filename)):
     # check the files which are end with specific extension
            # print path name of selected files
        if i == 2:
                break
        AUDIO_FILE = os.path.join(filename, file)
        waveform, sample_rate = torchaudio.load(AUDIO_FILE)
        waveform = waveform.to(device)
        

: 

In [7]:
AUDIO_FILE = r'/Users/shraymathur/MyFolders/UT/Research/data/fewshot_trn_audio'
embeddings = extract_audio_feat(AUDIO_FILE)

File  1
File  2
File  3
File  4
File  5
File  6
File  7
File  8
File  9
File  10
File  11
File  12
File  13
File  14
File  15
File  16
File  17
File  18
File  19
File  20
File  21
File  22
File  23
File  24
File  25
File  26
File  27
File  28
File  29
File  30
File  31
File  32
File  33
File  34
File  35
File  36
File  37
File  38
File  39
File  40
File  41
File  42
File  43
File  44
File  45
File  46
File  47
File  48
File  49
File  50
File  51
File  52
File  53
File  54
File  55
File  56
File  57
File  58
File  59
File  60
File  61
File  62
File  63
File  64
File  65
File  66
File  67
File  68
File  69
File  70
File  71
File  72
File  73
File  74
File  75
File  76
File  77
File  78
File  79
File  80
File  81
File  82
File  83
File  84
File  85
File  86
File  87
File  88
File  89
File  90
File  91
File  92
File  93
File  94
File  95
File  96
File  97
File  98
File  99
File  100
File  101
File  102
File  103
File  104
File  105
File  106
File  107
File  108
File  109
File  110
File  11

In [8]:
len(embeddings)

440

In [9]:
import pandas as pd
fewshot_trn_labels = pd.read_csv('/Users/shraymathur/MyFolders/UT/Sem2/ACV/fewshot-microvideo-recognition-main/data/fewshot_trn_labels.csv')
# fewshot_trn_labels[fewshot_trn_labels['PostIndex'] == 1010].iloc[0]['Topic']

filename = r'/Users/shraymathur/MyFolders/UT/Research/data/fewshot_trn_audio'
y_trn = []
for file in os.listdir(filename):
    postindex = int(file[0:4])
    topic = fewshot_trn_labels[fewshot_trn_labels['PostIndex'] == postindex].iloc[0]['Topic']
    y_trn.append(topic)


In [11]:
filenames = []
for file in os.listdir(filename):
     # check the files which are end with specific extension
#     if file.endswith(".wav"):
    filenames.append(file)

In [14]:
embeddings_dict = {"embeddings": embeddings, "y_trn": y_trn, "filenames": filenames}
embedding_filename = os.path.join("../data/embeddings/{}_{}_{}_embeddings.pt".format('audio', 'trn', '2'))
torch.save(embeddings_dict, embedding_filename)